In [1]:
!pip install -U requests
import requests

In [2]:
# 0. import modules that will be used here
import requests
import json
import time as time

# 1. Set the token endpoint URL
token_url = "https://sierra-test.cincinnatilibrary.org/iii/sierra-api/v6/token"

# 2. Set the client ID and secret obtained from the API provider
with open('config.json', 'r') as f:
  config = json.loads(f.read())
  client_id = config['client_id']
  client_secret = config['client_secret']

In [3]:
# 3. Set up a Requests Session object
session = requests.Session()

# 4. Set the default session headers
session.headers = {
    'accept': 'application/json',
    'Authorization': '',
}
# .. and define a session cookie to track when the token expires
# (expires_at is an interger "timestamp" --seconds since UNIX Epoch
session.cookies['expires_at'] = '0'

In [4]:
# 5. Define a function to authenticate our API requests
"""
https://gist.github.com/rayvoelker/7791c50f2176ae32799edd25431fb9db
"""
def authenticate_session(session):
  """
  Function, authenticate_session() accpepts a Requests Session object, 
  and then returns the session with headers containing a token that is 
  valid to make futue requests using the token
  """
  if (
      session.headers['Authorization'] == '' 
      or float(session.cookies['expires_at']) < time.time()
  ):  
    # Set the request parameters (for authentication and
    # the grant type)
    auth = requests.auth.HTTPBasicAuth(
      username=client_id, 
      password=client_secret
    )
    data = {
      "grant_type": "client_credentials"
    }

    # Send the token request
    # **this is Authorization Step 1**
    response = session.post(url=token_url, auth=auth, data=data)

    # Check if the request was successful
    if response.status_code == 200:
      # Extract the access token and expiration time from the response
      # **this is Authorization Step 2**
      session.headers['Authorization'] = \
        'Bearer ' + response.json()['access_token']

      # pad our expiration time by -60 seconds to be safe ...
      session.cookies['expires_at'] = \
        str(time.time() + response.json()['expires_in'] - 60)
      return(session)
    else:
      # If the request failed, raise an exception
      raise Exception('Failed to obtain access token: ' + response.text)

  else:
    # session, and token are still valid 
    return session

In [5]:
%%time
# **Make an authorized request to the v6/info/token endpoint

# authticate the session (if needed)
session = authenticate_session(session)

# make the request--/v6/info/token
response = session.get(
  'https://sierra-test.cincinnatilibrary.org/iii/sierra-api/v6/info/token'
)

print(f"""\
  response status code                  :{response.status_code}
  response json value for key expiresIn :{response.json()['expiresIn']}
  session expires at (UNIX Epoch)       :{session.cookies['expires_at']}
  time now                              :{time.time()}
  seconds left                          :{float(session.cookies['expires_at']) - time.time()}
""")

  response status code                  :200
  response json value for key expiresIn :3599
  session expires at (UNIX Epoch)       :1687791139.6931758
  time now                              :1687787599.7125504
  seconds left                          :3539.9805772304535

CPU times: user 21.4 ms, sys: 0 ns, total: 21.4 ms
Wall time: 69.2 ms


In [6]:
%%time

import time
import csv

# read the pNumbers from the file
with open('patrons.txt') as f:
    reader = csv.reader(f)
    # pnumbers in the file begin with a p, and end with a check digit, so remove them
    for i, record_num in enumerate([row[0][1:-1] for row in reader]):
        # normally these would all be different pickup locations per patron in the list
        body = {
          "recordType": "b",
          "recordNumber": 3538218,
          "pickupLocation": "dp",
          "note": "2023-06-26_test"
        }

        start = time.time()
        session = authenticate_session(session)
        result = session.post(f'https://sierra-test.cincinnatilibrary.org:443/iii/sierra-api/v6/patrons/{record_num}/holds/requests', json=body)
        print(i, 'record_num:', record_num, 'result:', result.status_code, 'time:', time.time() - start)

0 record_num: 1051812 result: 204 time: 0.6546077728271484
1 record_num: 1047610 result: 204 time: 0.6116688251495361
2 record_num: 1053267 result: 204 time: 0.6196732521057129
3 record_num: 1058058 result: 204 time: 0.6417458057403564
4 record_num: 1058252 result: 204 time: 0.5947918891906738
5 record_num: 1060234 result: 204 time: 0.6539695262908936
6 record_num: 1059049 result: 204 time: 0.6379523277282715
7 record_num: 1063233 result: 204 time: 0.5928270816802979
8 record_num: 1066451 result: 204 time: 0.6201112270355225
9 record_num: 1222108 result: 204 time: 0.6245179176330566
10 record_num: 1222366 result: 204 time: 0.5954833030700684
11 record_num: 1223191 result: 204 time: 0.5991346836090088
12 record_num: 1223672 result: 204 time: 0.6121354103088379
13 record_num: 1223695 result: 204 time: 0.592505931854248
14 record_num: 1226761 result: 204 time: 0.6406188011169434
15 record_num: 1205198 result: 204 time: 0.6437149047851562
16 record_num: 1209080 result: 204 time: 0.63741421